# Energy Load Data Cleaning Explanation

This notebook descrbes the process used to construct and clean the dataset.

Data was aquired from entsoe Transparency Platform at the following [link](https://transparency.entsoe.eu/load-domain/r2/totalLoadR2/show?name=&defaultValue=false&viewType=TABLE&areaType=BZN&atch=false&dateTime.dateTime=09.08.2015+00:00|CET|DAY&biddingZone.values=CTY|10YES-REE------0!BZN|10YES-REE------0&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2)#) (2015 data). Data is downloadable on an annual basis. this workbook constructs an example dataset using the years 2016-2018. The same functions may be used to construct any number of years available from this source.

Processes completed in the following functions:
1. format_data
    - renames the columns
    - shortens the text identifier for times
    - converts to a Datetime index
2. combine_annual_data
    - joins a dictionary if dataframes
3. interpolate_nans
    - fills the missing values using a linear interpolation method


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
!ls -l ./data/raw_data_ES

total 6776
-rw-r--r--@ 1 ns  staff  473123 18 Aug 20:03 Total Load - Day Ahead _ Actual_2015.csv
-rw-r--r--@ 1 ns  staff  474463 18 Aug 20:03 Total Load - Day Ahead _ Actual_2016.csv
-rw-r--r--@ 1 ns  staff  473173 18 Aug 20:03 Total Load - Day Ahead _ Actual_2017.csv
-rw-r--r--@ 1 ns  staff  473173 18 Aug 20:00 Total Load - Day Ahead _ Actual_2018.csv
-rw-r--r--@ 1 ns  staff  447309 18 Aug 20:00 Total Load - Day Ahead _ Actual_2019.csv
-rw-r--r--  1 ns  staff  946953 20 Aug 19:42 load_forecast_2016_2018.csv


In [33]:
path = './data/raw_data_ES/'
files = ['Total Load - Day Ahead _ Actual_2015.csv',
            'Total Load - Day Ahead _ Actual_2016.csv',
            'Total Load - Day Ahead _ Actual_2017.csv',
            'Total Load - Day Ahead _ Actual_2018.csv',
            'Total Load - Day Ahead _ Actual_2019.csv']

In [34]:
#load in list of the datasets
data_sets = [pd.read_csv(path+file) for file in files]

In [35]:
#inital look at the unprocessed data
data_sets[0].head(24)

,Time (CET),Day-ahead Total Load Forecast [MW] - BZN|ES,Actual Total Load [MW] - BZN|ES
0,01.01.2015 00:00 - 01.01.2015 01:00,26118.0,25385.0
1,01.01.2015 01:00 - 01.01.2015 02:00,24934.0,24382.0
2,01.01.2015 02:00 - 01.01.2015 03:00,23515.0,22734.0
3,01.01.2015 03:00 - 01.01.2015 04:00,22642.0,21286.0
4,01.01.2015 04:00 - 01.01.2015 05:00,21785.0,20264.0
5,01.01.2015 05:00 - 01.01.2015 06:00,21441.0,19905.0
6,01.01.2015 06:00 - 01.01.2015 07:00,21285.0,20010.0
7,01.01.2015 07:00 - 01.01.2015 08:00,21545.0,20377.0
8,01.01.2015 08:00 - 01.01.2015 09:00,21443.0,20094.0
9,01.01.2015 09:00 - 01.01.2015 10:00,21560.0,20637.0


### Formatting data

In [36]:
def format_data(data):
    '''
    Input: A dataframe of Day Ahead Total Load, and Actual Load obtained from csv data obtained from the entsoe Transparency Platform.
    
    Descrption:
    Input is a 3 column dataframe consisting of text time stamps with hourly frequency. 
    - Function formats the string in order to be formatted into a datetime.
    - Appends a datetime index and drops the time strings
    
    Output: A 2 column dataframe with a DatetimeIndex
    
    '''
    
    #set column names to something simple
    data.columns = ['time', 'day_forecast',
       'actual_load']

    #set the time to the first element in the time string. 
    #So 01.01.2018 00:00 - 01.01.2018 01:00 becomes 01.01.2018 00:00
    data['time'] = data['time'].str.split('-').apply(lambda x: x[0]).str.strip()
     
    #set the time strings to datetime obejects and set index as date time
    datetimes = pd.to_datetime(data['time'], format='%d-%m-%Y %H%M', errors='ignore')
    data_ = data.set_index(pd.DatetimeIndex(datetimes))
    
    #remove extra time column with original string objects
    data_time = data_[['day_forecast', 'actual_load']]
    
    return data_time

In [37]:
years = range(len(files))

#create a dictionary of formatted pandas dataframes where key is each year
format_sets = {year: format_data(data_set) for year,data_set in zip(years, data_sets)}


In [38]:
#take a look at the formatted data
format_sets[1].head()

,day_forecast,actual_load
time,,
2016-01-01 00:00:00,23273.0,22431.0
2016-01-01 01:00:00,22495.0,21632.0
2016-01-01 02:00:00,21272.0,20357.0
2016-01-01 03:00:00,20022.0,19152.0
2016-01-01 04:00:00,19148.0,18310.0


### Combine data into single dataframe

In [10]:
def combine_annual_data(dictionary):
    """
    Input: a dictionary of dataframes.
    
    Output: a single dataframe
    """
    
    all_data_list = []
    
    for key in dictionary.keys():
        all_data_list.append(dictionary[key])
        
    data_all_years = pd.concat(all_data_list)
    
    return data_all_years

In [39]:
#combine into one single dataframe
data = combine_annual_data(format_sets)

### Clean NANs

This data will be used for predicting day ahead energy demand. In dealing with nan values it is important not to change the structure of the data. 

Two ways this can occur:
   1. dropping values changes number of observations in a day. number of daily observations per day needs to line up with the days before and after.
   2. filling missing values with a single value (i.e. series mean value) is not representiative of the temporal nature of the data

In [40]:
#check for nans in the data
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 43829 entries, 2015-01-01 00:00:00 to 2019-12-31 23:00:00
Data columns (total 2 columns):
day_forecast    43819 non-null object
actual_load     43772 non-null object
dtypes: object(2)
memory usage: 1.0+ MB


Note that the data is type object. This likely indicates that there are string objects in the columns. These will have to be converted to nan before nans can be filled.

In [41]:
#count the total nans in each column and the total length of the data.
data.isnull().sum(), len(data)

(day_forecast    10
 actual_load     57
 dtype: int64, 43829)

A total of 3 and 12 nan values respectively in a dataset of length 26307. This is very clean data.

Can savfely imput the data with a linear interpolation function without changing the structure of the distribution.

In [42]:
#isolate the row indexes that with Nans
#use this to check the repalce_nans function works
nan_load = data[data['actual_load'].isnull()==True].index
nan_forecast = data[data['day_forecast'].isnull()==True].index
(nan_load, nan_forecast)

(DatetimeIndex(['2015-02-01 07:00:00', '2015-02-01 08:00:00',
                '2015-02-01 09:00:00', '2015-02-01 12:00:00',
                '2015-02-01 13:00:00', '2015-02-01 14:00:00',
                '2015-02-01 15:00:00', '2015-02-01 16:00:00',
                '2015-02-01 17:00:00', '2015-02-01 18:00:00',
                '2015-02-01 19:00:00', '2015-01-28 13:00:00',
                '2015-05-02 10:00:00', '2015-10-02 08:00:00',
                '2015-10-02 11:00:00', '2015-12-02 09:00:00',
                '2015-03-29 02:00:00', '2015-04-16 09:00:00',
                '2015-04-20 08:00:00', '2015-04-23 21:00:00',
                '2015-01-05 12:00:00', '2015-01-05 13:00:00',
                '2015-01-05 14:00:00', '2015-01-05 15:00:00',
                '2015-01-05 16:00:00', '2015-01-05 17:00:00',
                '2015-04-05 03:00:00', '2015-05-29 03:00:00',
                '2016-03-27 02:00:00', '2016-04-25 05:00:00',
                '2016-04-25 07:00:00', '2016-07-09 22:00:00',
        

In [61]:
def interpolate_nans(data):
    """
    Inputs:
    - data --- a dataframe of timeseries data
    - columns --- a list of column header names
    
    Process:
    Applies linear interpolation to fill the missing entries per column
    
    output: a dataframe
    """

    try:
        data = data.astype(float)
    except:
        for char in ['-', '--', '?']:
            data = data.replace('-', np.nan)
        data = data.astype(float)
    
    data = data.interpolate(method='linear', axis=0)
    
    return data

In [62]:
data = interpolate_nans(data)

#check the function works
data[data['actual_load'].isnull()==True]

,day_forecast,actual_load
time,,


In [63]:
#also see that the columns are now floats
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 40780 entries, 2015-01-01 00:00:00 to 2019-08-12 23:00:00
Data columns (total 2 columns):
day_forecast    40780 non-null float64
actual_load     40780 non-null float64
dtypes: float64(2)
memory usage: 955.8 KB


In [64]:
#take a look at the interpreted values and see how they compare to the values around them.
for t in nan_load:
    print(data[str(t)])

                     day_forecast  actual_load
time                                          
2015-02-01 07:00:00       24379.0      24710.5
                     day_forecast  actual_load
time                                          
2015-02-01 08:00:00       27389.0      27268.0
                     day_forecast  actual_load
time                                          
2015-02-01 09:00:00       30619.0      29825.5
                     day_forecast   actual_load
time                                           
2015-02-01 12:00:00       31357.0  32535.444444
                     day_forecast   actual_load
time                                           
2015-02-01 13:00:00       31338.0  32712.888889
                     day_forecast   actual_load
time                                           
2015-02-01 14:00:00       30874.0  32890.333333
                     day_forecast   actual_load
time                                           
2015-02-01 15:00:00       30124.0  33067.777778
 

### Check for duplicated timestamps

Because we are working with sequence data, it is important that there are the correct number of values per 24 hour period. If not, the data could at some point offset and become a source of error for the model.

In this case each day is 24 hours, and contains 24 readings. Therefore we can calculate how many data points we should have in a given period.

I.e. for the 5 year period cleaned in this example we have the years 2015, 2017, 2018, 2019 as non leap years, while 2016 is, so we have to account for it. Therefore (365 x 2 + 366) x 24 = 26304 hours.

In [20]:
print(len(data)==26304)
print(len(data))

False
43829


Therefore we have 3 duplicated entries. Drop the extra values and take the first occurance of the data point by default.

In [46]:
def remove_duplicated_rows(data):
    """
    Input a timeseries dataset with multiple rows of the same index value.
    
    Output timeseries dataset with first occurance of duplicated rows.
    """
    #identify the duplicated elements
    duplicated_rows_bool = data.index.duplicated()
    
    #invert the array element wise
    keep_rows = np.invert(duplicated_rows_bool)
    
    #return the original dataframe removing the duplicated values
    return data[keep_rows]

In [48]:
data = remove_duplicated_rows(data)
len(data)

26304

The data now lines up with the expected number of observations and may be exported.

In [51]:
#export the prepared data as csv
save_path = './data/processed_data/'
data.to_csv(save_path + 'load_forecast_2016_2018.csv')